### **Carga de datos desde AWS S3 a AWS RDS**

In [1]:
import pandas as pd
import boto3
import psycopg2
import io
from io import StringIO
import pyarrow
import pyarrow.parquet as pq
import sys

### Conexion y lectura de fuentes para procesamiento desde AWS S3



In [2]:
session = boto3.Session(
    aws_access_key_id='acces_key',
    aws_secret_access_key='secret_key',
)
client_s3 = session.client('s3')
resource_s3 = session.resource('s3')

### Lectura de parquet desde s3

In [8]:
buffer_c = io.BytesIO()

obj_comercio_parquet = resource_s3.Object('test-nequi-fraud','fraud-data/datos_comercio.parquet.gzip')
obj_comercio_parquet.download_fileobj(buffer_c)
df_comercio_parquet= pd.read_parquet(buffer_c)

print(df_comercio_parquet.head())

                  numero_trasaccion              nombre_local_comercial  \
0  0b242abb623afc578575680df30655b9          fraud_Rippin, Kub and Mann   
1  1f76529f8574734946361c461b024d99     fraud_Heller, Gutmann and Zieme   
2  a1a22d70485983eac12b5b88dad1cf95                fraud_Lind-Buckridge   
3  6b849c168bdad6f867558c3793159a81  fraud_Kutch, Hermiston and Farrell   
4  a41d7549acf90789359a9aa5346dcb46                 fraud_Keeling-Crist   

  categoria_local_comercial  latitud_local_comercial  longitud_local_comercial  
0                  misc_net                36.011293                -82.048315  
1               grocery_pos                49.159047               -118.186462  
2             entertainment                43.150704               -112.154481  
3             gas_transport                47.034331               -112.561071  
4                  misc_pos                38.674999                -78.632459  


In [3]:
buffer_f= io.BytesIO()

obj_fraude_parquet = resource_s3.Object('test-nequi-fraud','fraud-data/datos_fraude.parquet.gzip')
obj_fraude_parquet.download_fileobj(buffer_f)
df_fraude_parquet= pd.read_parquet(buffer_f)

print(df_fraude_parquet.head())

                  numero_trasaccion  fraude
0  0b242abb623afc578575680df30655b9       0
1  1f76529f8574734946361c461b024d99       0
2  a1a22d70485983eac12b5b88dad1cf95       0
3  6b849c168bdad6f867558c3793159a81       0
4  a41d7549acf90789359a9aa5346dcb46       0


### Lectura de CSV desde s3

In [43]:
obj_titulares_csv = client_s3.get_object(Bucket= 'test-nequi-fraud', Key= 'fraud-data/datos_titulares.csv')  
df_titulares_csv = pd.read_csv(obj_titulares_csv['Body'], sep=";")

print(df_titulares_csv.head())

                  numero_trasaccion nombre_titular_cuenta  \
0  0b242abb623afc578575680df30655b9              Jennifer   
1  1f76529f8574734946361c461b024d99             Stephanie   
2  a1a22d70485983eac12b5b88dad1cf95                Edward   
3  6b849c168bdad6f867558c3793159a81                Jeremy   
4  a41d7549acf90789359a9aa5346dcb46                 Tyler   

  apellido_titular_cuenta genero_titular_cuenta          calle_titular_cuenta  \
0                   Banks                     F                561 Perry Cove   
1                    Gill                     F  43039 Riley Greens Suite 393   
2                 Sanchez                     M      594 White Dale Suite 530   
3                   White                     M   9443 Cynthia Court Apt. 038   
4                  Garcia                     M              408 Bradley Rest   

  ciudad_titular_cuenta estado_titular_cuenta  codigo_postal_titular_cuenta  \
0        Moravian Falls                    NC                      

In [10]:
obj_transaccion_csv = client_s3.get_object(Bucket= 'test-nequi-fraud', Key= 'fraud-data/datos_transaccion.csv')  
df_transaccion_csv = pd.read_csv(obj_transaccion_csv['Body'], sep=";") 

print(df_transaccion_csv.head())

                  numero_trasaccion fecha_hora_transaccion  \
0  0b242abb623afc578575680df30655b9    2019-01-01 00:00:18   
1  1f76529f8574734946361c461b024d99    2019-01-01 00:00:44   
2  a1a22d70485983eac12b5b88dad1cf95    2019-01-01 00:00:51   
3  6b849c168bdad6f867558c3793159a81    2019-01-01 00:01:16   
4  a41d7549acf90789359a9aa5346dcb46    2019-01-01 00:03:06   

   numero_tarjeta_credito_transaccion  valor_transaccion  
0                    2703186189652095               4.97  
1                        630423337322             107.23  
2                      38859492057661             220.11  
3                    3534093764340240              45.00  
4                     375534208663984              41.96  


------

**CONEXIÓN Y TRABAJO CON AWS RDS**

Funciones de conexion e insert en rds

In [4]:
def connect_psy():
    engine = None
    try:
    # Conecta a PostgreSQL server
        print('Conectando a aws PostgreSQL database...')
        engine = psycopg2.connect(
          database="db",
          user="user",
          password="password",
          host="xxxxxxxxxxxxxxxxxxxxxx-1.rds.amazonaws.com",
          port='5432')
        print('Conecto a aws PostgreSQL database...')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return engine

In [36]:
def copy_from_stringio(engine, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    
    #numero_transaccion como index para primary key
    df.set_index('numero_trasaccion', inplace=True)
    df.to_csv(buffer, index_label='', header=False)
    
    buffer.seek(0)
    cursor = engine.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        engine.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        
        print("Error: %s" % error)
        engine.rollback() #Revierte insert
        cursor.close()
        print(df.head(1))
        return 1
    
    print("copy_from_stringio() done")
    
    cursor.close()

### Insertar df_fraude_parquet

Se realizar carga en lotes de 10000 registros.

##### Carga de datos a la tabla de datos_fraude

In [7]:
# Inicio Conexion
engine = connect_psy()

# Carga de datos en lotes de 10.000 registros para optimizacion de memoria 
for i in range(1,len(df_fraude_parquet),10000):
    resta= len(df_fraude_parquet) - i
    
    if resta > 10000:
        limit = 10000-1
    else:
        limit = resta+1
        
    df_fraude_parquet_s = df_fraude_parquet[i-1:i+limit]
    
    print("generando copy para registros entre {} y {}".format((i-1),(i+limit)))
    copy_from_stringio(engine, df_fraude_parquet_s, 'datos_fraude')
    
    if copy == 1: break # Si existe error al copy se detiene el proceso
        
# Cierra Conexion
engine.close()

Conectando a aws PostgreSQL database...
Conecto a aws PostgreSQL database...
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_str

##### Carga de datos a la tabla de datos_comercio

In [48]:
# Inicio Conexion
engine = connect_psy()

# Carga de datos en lotes de 10.000 registros para optimizacion de memoria 
for i in range(1,len(df_comercio_parquet),10000):
    resta= len(df_comercio_parquet) - i
    
    if resta > 10000:
        limit = 10000-1
    else:
        limit = resta+1
       
    df_s = df_comercio_parquet[i-1:i+limit]
    # Reemeplazo de , po - por uso de , como separador nativo del dataframe
    df_s['nombre_local_comercial'] = df_s['nombre_local_comercial'].str.replace(',','-') 
    
    print("generando copy para registros entre {} y {}".format((i-1),(i+limit)))
    copy = copy_from_stringio(engine, df_s, 'datos_comercio')
    
    if copy == 1: break # Si existe error al copy se detiene el proceso
        
# Cierra Conexion
engine.close()

Conectando a aws PostgreSQL database...
Conecto a aws PostgreSQL database...


C:\Users\rmc_1\AppData\Local\Temp\ipykernel_7600\2514661705.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['nombre_local_comercial'] = df_s['nombre_local_comercial'].str.replace(',','-')


copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_stringio() done
copy_from_st

##### Carga de datos a la tabla de datos_titulares

In [49]:
# Inicio Conexion
engine = connect_psy()

# Carga de datos en lotes de 10.000 registros para optimizacion de memoria 
for i in range(1,len(df_titulares_csv),10000):
    resta= len(df_titulares_csv) - i
    
    if resta > 10000:
        limit = 10000-1
    else:
        limit = resta+1
        
    df_s = df_titulares_csv[i-1:i+limit]
    # Reemeplazo de , po - por uso de , como separador nativo del dataframe
    df_s['trabajo_titular_cuenta'] = df_s['trabajo_titular_cuenta'].str.replace(',','-')

    print("generando copy para registros entre {} y {}".format((i-1),(i+limit)))
    copy = copy_from_stringio(engine, df_s, 'datos_titulares')
    
    if copy == 1: break # Si existe error al copy se detiene el proceso
        
# Cierra Conexion
engine.close()

Conectando a aws PostgreSQL database...
Conecto a aws PostgreSQL database...
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 0 y 10000


C:\Users\rmc_1\AppData\Local\Temp\ipykernel_7600\3701790780.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['trabajo_titular_cuenta'] = df_s['trabajo_titular_cuenta'].str.replace(',','-')


copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 10000 y 20000
copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 20000 y 30000
copy_from_stringio() done
cantidad de columnas 13
nomb

copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 190000 y 200000
copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 200000 y 210000
copy_from_stringio() done
cantidad de columnas 13


copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 370000 y 380000
copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 380000 y 390000
copy_from_stringio() done
cantidad de columnas 13


copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 550000 y 560000
copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 560000 y 570000
copy_from_stringio() done
cantidad de columnas 13


copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 730000 y 740000
copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 740000 y 750000
copy_from_stringio() done
cantidad de columnas 13


copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 910000 y 920000
copy_from_stringio() done
cantidad de columnas 13
nombre de columnas ['numero_trasaccion', 'nombre_titular_cuenta', 'apellido_titular_cuenta', 'genero_titular_cuenta', 'calle_titular_cuenta', 'ciudad_titular_cuenta', 'estado_titular_cuenta', 'codigo_postal_titular_cuenta', 'latitud_titular_cuenta', 'longitud_titular_cuenta', 'poblacion_ciudad_titular_cuenta', 'trabajo_titular_cuenta', 'fecha_nacimiento_titular_cuenta']
generando copy para registros entre 920000 y 930000
copy_from_stringio() done
cantidad de columnas 13


##### Carga de datos a la tabla de datos_transaccion

In [17]:
# Inicio Conexion
engine = connect_psy()

# Carga de datos en lotes de 10.000 registros para optimizacion de memoria 
for i in range(1,len(df_transaccion_csv),10000):
    resta= len(df_transaccion_csv) - i
    
    if resta > 10000:
        limit = 10000-1
    else:
        limit = resta+1
        
    df_s = df_transaccion_csv[i-1:i+limit]
    
    print("generando copy para registros entre {} y {}".format((i-1),(i+limit)))
    copy = copy_from_stringio(engine, df_s, 'datos_transaccion')

    if copy == 1: break # Si existe error al copy se detiene el proceso
        
# Cierra Conexion
engine.close()

Conectando a aws PostgreSQL database...
Conecto a aws PostgreSQL database...
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
0b242abb623afc578575680df30655b9    2019-01-01 00:00:18   
1f76529f8574734946361c461b024d99    2019-01-01 00:00:44   
a1a22d70485983eac12b5b88dad1cf95    2019-01-01 00:00:51   
6b849c168bdad6f867558c3793159a81    2019-01-01 00:01:16   
a41d7549acf90789359a9aa5346dcb46    2019-01-01 00:03:06   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
0b242abb623afc578575680df30655b9                    2703186189652095   
1f76529f8574734946361c461b024d99                        630423337322   
a1a22d70485983eac12b5b88dad1cf95                      38859492057661   
6b849c168bdad6f867558c3793159a81                    3534093764340240   
a41d7549acf90789359a9aa5346dcb46                     375534208663984   

    

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
6dc0b18f1c57e929a40ed04400a63d7c    2019-02-11 05:58:41   
45d8e402010134c3acab2f32a36f456e    2019-02-11 05:58:45   
ecd9a933c28dcd80c273d675ac262e3b    2019-02-11 05:59:20   
38e2e0131b113b797234235f76c2f957    2019-02-11 05:59:40   
4c8cf2e9a1ba1eb11ea5a1e0f3eb4cc8    2019-02-11 06:00:10   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
6dc0b18f1c57e929a40ed04400a63d7c                    4430881574719618   
45d8e402010134c3acab2f32a36f456e                 4457732997086323466   
ecd9a933c28dcd80c273d675ac262e3b                    3528407217576457   
38e2e0131b113b797234235f76c2f957                        561942763351   
4c8cf2e9a1ba1eb11ea5a1e0f3eb4cc8                        630424987505   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
db541d1e8188954f55186b6b846e20cc    2019-03-17 13:33:24   
6b61bba7d874d8e32304c1143b5728b8    2019-03-17 13:33:37   
435f4bd199c8f92bf1b5961ef2b50723    2019-03-17 13:33:38   
00f6c898755207b63c90e5bd3538f18c    2019-03-17 13:33:40   
904cbeaa23db1c26dfd0acb15bf33932    2019-03-17 13:33:40   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
db541d1e8188954f55186b6b846e20cc                     346243940647414   
6b61bba7d874d8e32304c1143b5728b8                    3502088871723054   
435f4bd199c8f92bf1b5961ef2b50723                    4708992452821239   
00f6c898755207b63c90e5bd3538f18c                    3513618443244549   
904cbeaa23db1c26dfd0acb15bf33932                    4642255475285942   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
4949b5c19b98015ca19aef1c19437afa    2019-09-16 03:04:38   
8ade40a4b1224b4ba4a06a757f79ceda    2019-09-16 03:06:24   
02d7e256aa64ba89864777683473c130    2019-09-16 03:06:55   
4549ba9e35c5c70542598de592ecc8c1    2019-09-16 03:07:12   
50eba4b7ae08a512371e8496735874af    2019-09-16 03:07:47   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
4949b5c19b98015ca19aef1c19437afa                    4342532437704183   
8ade40a4b1224b4ba4a06a757f79ceda                        630469040731   
02d7e256aa64ba89864777683473c130                    3573030041201292   
4549ba9e35c5c70542598de592ecc8c1                    4658490815480264   
50eba4b7ae08a512371e8496735874af                    2229824890647120   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
19c6740a499d91d014917730c2ca2044    2019-10-17 05:55:38   
4f26233d1fd6f62b86212d5679008780    2019-10-17 05:56:47   
cee0751486621b90a46ca6cb50e1c0de    2019-10-17 05:58:06   
0755c07138d1668a47737e7358d543db    2019-10-17 05:58:22   
82cedf5190dafe85cc08da19e14cdb23    2019-10-17 05:58:38   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
19c6740a499d91d014917730c2ca2044                     376445266762684   
4f26233d1fd6f62b86212d5679008780                    4599285557366057   
cee0751486621b90a46ca6cb50e1c0de                    3538520143479972   
0755c07138d1668a47737e7358d543db                    4996263498048679   
82cedf5190dafe85cc08da19e14cdb23                    4003989662068504   

                                  valor_transaccion  
n

...

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
d6152d4ae076b8c50df5e439cce815fb    2020-05-03 18:07:07   
201edc993f26c342083e9bef10526b53    2020-05-03 18:09:26   
5c4dd62f37c7505f24d7ad80727b8881    2020-05-03 18:09:47   
56e53d10c53d973459579f27b318e1cd    2020-05-03 18:10:15   
48b1bdc88c7a1f50037a80c78c4e8440    2020-05-03 18:10:19   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
d6152d4ae076b8c50df5e439cce815fb                 4980323467523543940   
201edc993f26c342083e9bef10526b53                    3598634130286287   
5c4dd62f37c7505f24d7ad80727b8881                    4173950183554608   
56e53d10c53d973459579f27b318e1cd                    4430881574719618   
48b1bdc88c7a1f50037a80c78c4e8440                     372520049757633   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
4af09f4338a781ea8be9625e1b8c4f08    2020-06-01 08:15:43   
2a1da5bc846e1a1d70c9d9a0b7557a0b    2020-06-01 08:15:43   
1d52330a0c0386cfda47674d33f1c315    2020-06-01 08:15:48   
32fc569b29d5e81d0bb4cf48b85765a8    2020-06-01 08:15:52   
897c02c95b839e613a446de1cadf32db    2020-06-01 08:16:32   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
4af09f4338a781ea8be9625e1b8c4f08                    3582402971899057   
2a1da5bc846e1a1d70c9d9a0b7557a0b                       4623560839669   
1d52330a0c0386cfda47674d33f1c315                     374238209524200   
32fc569b29d5e81d0bb4cf48b85765a8                    3521417320836166   
897c02c95b839e613a446de1cadf32db                     374238209524200   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
b8e3513c8ba7e5f9f6665e486099d7ba    2020-06-25 17:58:32   
e764a4d5d94ebe394a0e12cd15b6fcd2    2020-06-25 18:01:27   
21402766bc34f17e7a22a8918d081580    2020-06-25 18:01:35   
f88aa85eea3438d3302c19984c3fa3e2    2020-06-25 18:01:37   
a7b3c06fec3d416bc112d9f8fb98079c    2020-06-25 18:02:02   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
b8e3513c8ba7e5f9f6665e486099d7ba                    3598014571045296   
e764a4d5d94ebe394a0e12cd15b6fcd2                     213154573301411   
21402766bc34f17e7a22a8918d081580                    3589255887819806   
f88aa85eea3438d3302c19984c3fa3e2                    3533800906065217   
a7b3c06fec3d416bc112d9f8fb98079c                    3564839259330465   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
9cce7acf35cf3519633a1ef4ebca2f34    2020-07-20 10:28:50   
05a768248d0f46c4c283fc98761750c1    2020-07-20 10:30:01   
34380044d777a07db9b652a5e4239646    2020-07-20 10:30:27   
4804ce9589870c684b1f1bb19064e8eb    2020-07-20 10:30:39   
fac3a9a993b817de2f19966cd53a1d74    2020-07-20 10:31:56   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
9cce7acf35cf3519633a1ef4ebca2f34                     180017442990269   
05a768248d0f46c4c283fc98761750c1                    4789911625179723   
34380044d777a07db9b652a5e4239646                    3586955669388457   
4804ce9589870c684b1f1bb19064e8eb                       4990494243023   
fac3a9a993b817de2f19966cd53a1d74                    2283743876903625   

                                  valor_transaccion  
n

copy_from_stringio() done
                                 fecha_hora_transaccion  \
numero_trasaccion                                         
247d6f47e257e6e6aaee20cd4d06cf30    2020-08-13 22:51:22   
f5060ce4c95a6613e9f6d844802546b5    2020-08-13 22:51:30   
09129f946b20d99b38ef2c5a23e0a6ed    2020-08-13 22:51:36   
1064b9c0e023d3f35ec61952b8ac890c    2020-08-13 22:51:54   
d8509b38f04c6c354e542b3dbb64af6a    2020-08-13 22:52:03   

                                  numero_tarjeta_credito_transaccion  \
numero_trasaccion                                                      
247d6f47e257e6e6aaee20cd4d06cf30                 4512828414983801773   
f5060ce4c95a6613e9f6d844802546b5                    4874006077381178   
09129f946b20d99b38ef2c5a23e0a6ed                       4427805710168   
1064b9c0e023d3f35ec61952b8ac890c                        583699482621   
d8509b38f04c6c354e542b3dbb64af6a                      38580485618059   

                                  valor_transaccion  
n